<a href="https://colab.research.google.com/github/nelsonalbertohj/Magno-Parvo-CNN/blob/main/Magno_parvo_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_addons

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import pandas as pd

import keras
import time
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Average
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
print("Tensorflow version: ",tf.__version__)

Tensorflow version:  2.8.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
root = "/content/drive/MyDrive/Vision Dataset/Imagenet-10-1500-splits/"
loc = root + "train"
data_dir = pathlib.Path(loc)

batch_size = 32
img_height = 224
img_width = 224

#Training set
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  label_mode = 'categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

#Testing set
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  label_mode = 'categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 11636 files belonging to 10 classes.
Using 9309 files for training.
Found 11636 files belonging to 10 classes.
Using 2327 files for validation.


In [6]:
def img_transforms(data,label):
    normalize_img = tf.keras.layers.Rescaling(1./255)
    color_norm = normalize_img(data)
    gray_img = tf.image.rgb_to_grayscale(data)
    gray_norm_img = normalize_img(gray_img)
    gray_norm_img = tfa.image.gaussian_filter2d(gray_norm_img,filter_shape=(10,10),sigma=5.0)
    concat_img = tf.concat([color_norm, gray_norm_img],axis=-1)
    return concat_img,label

AUTOTUNE = tf.data.AUTOTUNE

train_ds_preprocessed = train_ds.map(img_transforms)
val_ds_preprocessed = val_ds.map(img_transforms)

#NOTE: THE CASHE ON DISK IS ONLY NECESSARY IF HAVING ISSUES WITH IMAGE LOADING BOTTLENECK
CASHE_Train = f"{root}CASHE_Train_Blur_50_1500"
CASHE_Val = f"{root}CASHE_Val_Blur_50_1500"
train_ds_preprocessed = train_ds_preprocessed.cache(CASHE_Train).prefetch(buffer_size=AUTOTUNE)
val_ds_preprocessed = val_ds_preprocessed.cache(CASHE_Val).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Used to create CASHE in local disk by loading all images
idx = 0
start_time = time.time()
for t in train_ds_preprocessed:
  print("time to get object: ", time.time()-start_time)
  idx += 1
  start_time = time.time()

time to get object:  33.0582971572876
time to get object:  12.263034343719482
time to get object:  14.473114728927612
time to get object:  12.226741313934326
time to get object:  16.13148045539856
time to get object:  12.200576305389404
time to get object:  11.591894388198853
time to get object:  12.674214601516724
time to get object:  13.647250652313232
time to get object:  11.113381147384644
time to get object:  12.316364049911499
time to get object:  13.428166151046753
time to get object:  12.962493896484375
time to get object:  13.708700895309448
time to get object:  11.886165857315063
time to get object:  13.076969623565674
time to get object:  14.407016515731812
time to get object:  13.170894145965576
time to get object:  10.560872793197632
time to get object:  13.154829263687134
time to get object:  12.825519323348999
time to get object:  12.01262640953064
time to get object:  13.416741132736206
time to get object:  12.664159297943115
time to get object:  13.163422584533691
time

In [ ]:
class_names = train_ds.class_names
gray_imgs = []
plt.figure(figsize=(10, 10))
for images, labels in train_ds_preprocessed.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        current_img = images[i]
        plt.imshow(current_img.numpy()[:,:,:-1])
        gray_imgs.append((current_img,np.argmax(labels[i])))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")
plt.figure(figsize=(10, 10))
for images, labels in train_ds_preprocessed.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        current_img = images[i]
        plt.imshow(current_img.numpy()[:,:,-1],cmap="gray")
        gray_imgs.append((current_img,np.argmax(labels[i])))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")

In [ ]:
#Visualizing gray images and other effects
np.random.seed(0)

def img_transforms_no_blur(data,label):
    normalize_img = tf.keras.layers.Rescaling(1./255)
    color_norm = normalize_img(data)
    gray_img = tf.image.rgb_to_grayscale(data)
    gray_norm_img = normalize_img(gray_img)
    concat_img = tf.concat([color_norm, gray_norm_img],axis=-1)
    return concat_img,label

val_ds_preprocessed_no_blur = val_ds.map(img_transforms_no_blur)

class_names = train_ds.class_names
gray_imgs = []
plt.figure(figsize=(10, 10))
for images, labels in val_ds_preprocessed_no_blur.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        current_img = images[i]
        plt.imshow(current_img.numpy()[:,:,-1],cmap="gray")
        gray_imgs.append((current_img,np.argmax(labels[i])))
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")

plt.figure(figsize=(10, 10))
for i, (images, labels) in enumerate(gray_imgs):
      ax = plt.subplot(3, 3, i + 1)
      blurred_img = tfa.image.gaussian_filter2d(images[:,:,-1:],filter_shape=(10,10),sigma=5.0)
      plt.imshow(blurred_img.numpy().squeeze(),cmap="gray")
      plt.title(class_names[labels])
      plt.axis("off")

### Dual Stream ResNet Architecture

In [ ]:
from keras.backend import dropout
class DualStream_ResNet(tf.keras.Model):
    def __init__(self, n_outputs, batch_size, freeze = True, load_type=None, 
                 merge_type="averaging"):
        super().__init__()
        self.n_outputs =  n_outputs
        self.merge_type = merge_type

        self.magno_stream = tf.keras.applications.ResNet50(
                            include_top=False,
                            weights= load_type,
                            input_tensor=None,
                            input_shape=None,
                            pooling=None,
                            classes=n_outputs,
                            )
        
        self.parvo_stream = tf.keras.applications.ResNet50(
                            include_top=False,
                            weights= load_type,
                            input_tensor=None,
                            input_shape=None,
                            pooling=None,
                            classes=n_outputs,
                            )
        if freeze:
          self.magno_stream.trainable = False
          self.parvo_stream.trainable = False

        self.flat = tf.keras.layers.Flatten()
        self.fc = Sequential([#tf.keras.layers.Dense(2048,activation='ReLU'),
                              # tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(1280, activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(640, activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(self.n_outputs, activation='softmax')])
        
        self.avg = tf.keras.layers.Average()
        self.concat = tf.keras.layers.Concatenate()

    def call(self,inputs):
        color_input = inputs[:,:,:,:-1]
        gray_input = inputs[:,:,:,-1:]
        concat_gray_input = tf.concat([gray_input,
                                      gray_input,
                                      gray_input],axis=-1)

        m_stream = self.magno_stream(color_input)
        m_stream = self.flat(m_stream)
        
        p_stream = self.parvo_stream(concat_gray_input)
        p_stream = self.flat(p_stream)
        
        if self.merge_type == "averaging":
          avg_outputs = self.avg([p_stream,m_stream])
        elif self.merge_type == "concat":
          avg_outputs = self.concat([p_stream,m_stream])
        else:
          raise "That layer option is not available"
        return self.fc(avg_outputs)

In [ ]:
FREEZE = False 
LOAD_WEIGHTS = None #One of None or imagenet
dual_stream_model = DualStream_ResNet(10,batch_size,FREEZE,load_type=LOAD_WEIGHTS,
                                      merge_type="averaging")
dual_stream_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model_save_loc = "/content/drive/MyDrive/Vision Dataset/Models/"
model_name = "Dualstream_with_ResNet_1000_Samples_Deeper_10Classes"
checkpoint_path = f"{model_save_loc}{model_name}"
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_accuracy',
                                                 save_weights_only=True,
                                                 verbose=1)

train_history = dual_stream_model.fit(train_ds_preprocessed, validation_data= val_ds_preprocessed,
                                      epochs= 60, callbacks=[callback,cp_callback], workers = 4)

# dual_stream_model.save()

#Save training history to CSV
hist_df = pd.DataFrame(train_history.history)

hist_csv_file = f"{model_save_loc}{model_name}_history.csv"
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

plt.figure()
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Epoch 1/60
1502/1502 [==============================] - ETA: 0s - loss: 5.0708 - accuracy: 0.0409
Epoch 1: saving model to /content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Dataset/Models/Dualstream_with_ResNet_1000_Samples_Deeper_50Classes
1502/1502 [==============================] - 598s 391ms/step - loss: 5.0708 - accuracy: 0.0409 - val_loss: 3.7750 - val_accuracy: 0.0769
Epoch 2/60
1502/1502 [==============================] - ETA: 0s - loss: 3.3881 - accuracy: 0.1164
Epoch 2: saving model to /content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Dataset/Models/Dualstream_with_ResNet_1000_Samples_Deeper_50Classes
1502/1502 [==============================] - 586s 390ms/step - loss: 3.3881 - accuracy: 0.1164 - val_loss: 3.1850 - val_accuracy: 0.1691
Epoch 3/60
1502/1502 [==============================] - ETA: 0s - loss: 3.0746 - accuracy: 0.1818
Epoch 3: saving model to /content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Datas

KeyboardInterrupt: ignored

In [ ]:
#Load Model:
dual_stream_model = tf.keras.models.load_model("/content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Dataset/Models/Dualstream_with_ResNet_1000_Samples_Deeper_50Classes")
# eval_loss, eval_accuracy = dual_stream_model.evaluate(val_ds_preprocessed)


### Simple ResNet architectur test

In [ ]:
#Testing Pure ResNet Model Training
from keras.backend import dropout
class Single_ResNet(tf.keras.Model):
    def __init__(self, n_outputs, batch_size, freeze = True, load_type=None, 
                 merge_type="averaging"):
        super().__init__()
        self.n_outputs =  n_outputs
        self.merge_type = merge_type

        self.magno_stream = tf.keras.applications.ResNet50(
                            include_top=False,
                            weights= load_type,
                            input_tensor=None,
                            input_shape=None,
                            pooling=None,
                            classes=n_outputs,
                            )
        if freeze:
          self.magno_stream.trainable = False

        self.flat = tf.keras.layers.Flatten()
        self.fc = Sequential([#tf.keras.layers.Dense(2048,activation='ReLU'),
                              #tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(1280, activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(640, activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(self.n_outputs, activation='softmax')])
        
        self.avg = tf.keras.layers.Average()
        self.concat = tf.keras.layers.Concatenate()

    def call(self,inputs):
        color_input = inputs[:,:,:,:-1]

        m_stream = self.magno_stream(color_input)
        m_stream = self.flat(m_stream)

        return self.fc(m_stream)

In [ ]:
FREEZE = False 
LOAD_WEIGHTS = None #One of None or imagenet
signle_resnet_model = Single_ResNet(10,batch_size,FREEZE,load_type=LOAD_WEIGHTS,
                                      merge_type="averaging")
signle_resnet_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)

train_history = signle_resnet_model.fit(train_ds_preprocessed, validation_data= val_ds_preprocessed,
                                      epochs= 30, callbacks=callback, workers = 4)
signle_resnet_model.save("/content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Dataset/Models/Single_ResNet_1000_Samples_Deeper")

Epoch 1/30
291/291 [==============================] - 72s 227ms/step - loss: 13.6862 - accuracy: 0.1110 - val_loss: 2.3041 - val_accuracy: 0.1156
Epoch 2/30
291/291 [==============================] - 64s 219ms/step - loss: 2.2606 - accuracy: 0.1448 - val_loss: 2.2146 - val_accuracy: 0.1586
Epoch 3/30
291/291 [==============================] - 64s 219ms/step - loss: 2.2257 - accuracy: 0.1625 - val_loss: 2.1758 - val_accuracy: 0.1844
Epoch 4/30
291/291 [==============================] - 63s 216ms/step - loss: 2.1793 - accuracy: 0.1795 - val_loss: 2.1855 - val_accuracy: 0.1624
Epoch 5/30
291/291 [==============================] - 64s 220ms/step - loss: 2.1579 - accuracy: 0.1872 - val_loss: 2.0857 - val_accuracy: 0.1990
Epoch 6/30
291/291 [==============================] - 64s 220ms/step - loss: 2.1143 - accuracy: 0.2057 - val_loss: 2.0430 - val_accuracy: 0.2119
Epoch 7/30
291/291 [==============================] - 64s 219ms/step - loss: 2.0276 - accuracy: 0.2475 - val_loss: 1.8438 - val_a

INFO:tensorflow:Assets written to: /content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Dataset/Models/Single_ResNet_1000_Samples_Deeper/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Vision Dataset/Models/Single_ResNet_1000_Samples_Deeper/assets


### DualStream with VGG versions (Pending)

### DualStream EfficientNet Architecture
(limitation: it overfits really badly on our current dataset. The literature reports overfitting when smaller training sets are given as a serious issue with EffectiveNet)

In [ ]:
import tensorflow as tf
                                                                                
class DualStream_ENet(tf.keras.Model):
    def __init__(self, n_outputs, batch_size, freeze = True, load_type=None,
                 merge_type="averaging"):
        super().__init__()
        self.n_outputs =  n_outputs
        self.merge_type = merge_type

        self.magno_stream = tf.keras.applications.EfficientNetB0(
                            include_top=False,
                            weights= load_type,
                            input_tensor=None,
                            input_shape=None,
                            pooling=None,
                            classes=self.n_outputs,
                            classifier_activation="softmax",
                            )
        
        self.parvo_stream = tf.keras.applications.EfficientNetB0(
                            include_top=False,
                            weights= load_type,
                            input_tensor=None,
                            input_shape=None,
                            pooling=None,
                            classes=self.n_outputs,
                            classifier_activation="softmax",
                            )
        if freeze:
          self.magno_stream.trainable = False
          self.parvo_stream.trainable = False

        self.flat = tf.keras.layers.Flatten()
        self.fc = Sequential([tf.keras.layers.Dense(2048,activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(1280, activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(640, activation='ReLU'),
                              tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(self.n_outputs, activation='softmax')])
        
        self.avg = tf.keras.layers.Average()
        self.concat = tf.keras.layers.Concatenate()
        self.fc_out = tf.keras.layers.Dense(self.n_outputs, activation='softmax')

    def call(self,inputs):
        color_input = inputs[:,:,:,:-1]
        # start_time = time.time()
        gray_input = inputs[:,:,:,-1:]
        concat_gray_input = tf.concat([gray_input,
                                      gray_input,
                                      gray_input],axis=-1)
        # print("delta_time:",time.time()-start_time)

        m_stream = self.magno_stream(color_input)
        m_stream = self.flat(m_stream)
        
        p_stream = self.parvo_stream(concat_gray_input)
        p_stream = self.flat(p_stream)
        
        if self.merge_type == "averaging":
          avg_outputs = self.avg([p_stream,m_stream])
        elif self.merge_type == "concat":
          avg_outputs = self.concat([p_stream,m_stream])
        else:
          raise "That layer option is not available"
        return self.fc(avg_outputs)

In [ ]:
FREEZE = False #One of None or imagenet
dual_stream_model = DualStream_ENet(50,batch_size,FREEZE,load_type="imagenet",
                                    merge_type="averaging")
dual_stream_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# checkpoint_path = "/content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Models//content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Models/Dualstream_with_ResNet2.0"
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

train_history = dual_stream_model.fit(train_ds_preprocessed, validation_data= val_ds_preprocessed,
                                      epochs= 30, callbacks=callback, workers = 4)
dual_stream_model.save("/content/drive/MyDrive/School/MIT Spring2022/9.60/Vision Project/Models/Dualstream_with_EfficientNet_Deeper")

Epoch 1/30
489/489 [==============================] - 348s 671ms/step - loss: 2.9101 - accuracy: 0.3179 - val_loss: 5.7237 - val_accuracy: 0.0133
Epoch 2/30
489/489 [==============================] - 325s 663ms/step - loss: 2.3374 - accuracy: 0.4244 - val_loss: 19.3927 - val_accuracy: 0.0190
Epoch 3/30
489/489 [==============================] - 325s 663ms/step - loss: 2.2102 - accuracy: 0.4646 - val_loss: 44.7883 - val_accuracy: 0.0207
Epoch 4/30
489/489 [==============================] - 325s 664ms/step - loss: 2.2732 - accuracy: 0.4535 - val_loss: 13.0702 - val_accuracy: 0.0228
Epoch 5/30
489/489 [==============================] - 324s 662ms/step - loss: 2.1662 - accuracy: 0.4679 - val_loss: 4.0507 - val_accuracy: 0.0251
Epoch 6/30
489/489 [==============================] - 324s 661ms/step - loss: 2.1699 - accuracy: 0.4697 - val_loss: 136.9688 - val_accuracy: 0.0182
Epoch 7/30
489/489 [==============================] - 324s 662ms/step - loss: 2.3259 - accuracy: 0.4434 - val_loss: 5.0

### Dual Stream VGG Net

In [ ]:
class DualStream_VGG(tf.keras.Model):
    def __init__(self, n_outputs, batch_size, merge_type="averaging"):
        super().__init__()
        self.n_outputs =  n_outputs
        self.merge_type = merge_type

        self.magno_stream = Sequential([
        Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Flatten()
        ])
        
        self.parvo_stream = Sequential([
        Conv2D(input_shape=(224,224,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
        MaxPool2D(pool_size=(2,2),strides=(2,2)),
        Flatten()
        ])

        self.flat = tf.keras.layers.Flatten()
        self.fc = Sequential([#tf.keras.layers.Dense(2048,activation='ReLU'),
                              # tf.keras.layers.Dropout(0.5),
                              # tf.keras.layers.Dense(1280, activation='ReLU'),
                              # tf.keras.layers.Dropout(0.5),
                              # tf.keras.layers.Dense(640, activation='ReLU'),
                              # tf.keras.layers.Dropout(0.5),
                              tf.keras.layers.Dense(self.n_outputs, activation='softmax')])
        
        self.avg = tf.keras.layers.Average()
        self.concat = tf.keras.layers.Concatenate()

    def call(self,inputs):
        color_input = inputs[:,:,:,:-1]
        gray_input = inputs[:,:,:,-1:]

        m_stream = self.magno_stream(color_input)
        m_stream = self.flat(m_stream)
        
        p_stream = self.parvo_stream(gray_input)
        p_stream = self.flat(p_stream)
        
        if self.merge_type == "averaging":
          avg_outputs = self.avg([p_stream,m_stream])
        elif self.merge_type == "concat":
          avg_outputs = self.concat([p_stream,m_stream])
        else:
          raise "That layer option is not available"
        return self.fc(avg_outputs)

In [ ]:
dual_stream_model = DualStream_VGG(10,batch_size,merge_type="averaging")
dual_stream_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model_save_loc = "/content/drive/MyDrive/Vision Dataset/Models/"
model_name = "Dualstream_with_VGG_1000_Samples_10Classes"
checkpoint_path = f"{model_save_loc}{model_name}"
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_accuracy',
                                                 save_weights_only=True,
                                                 verbose=1)

train_history = dual_stream_model.fit(train_ds_preprocessed, validation_data= val_ds_preprocessed,
                                      epochs= 60, callbacks=[callback,cp_callback], workers = 4)

# dual_stream_model.save()

#Save training history to CSV
hist_df = pd.DataFrame(train_history.history)

hist_csv_file = f"{model_save_loc}{model_name}_history.csv"
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

plt.figure()
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()